In [1]:
import xarray as xr
import datetime
import numpy as np
from metpy.calc import relative_humidity_from_specific_humidity
from metpy.units import units
from pyproj import Proj, Transformer, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import rasterio
from shapely.geometry import Point
import geopandas as gpd
import os
import glob
from zipfile import ZipFile
import pandas as pd
import matplotlib.pylab as plt
import shapely
import xesmf as xe

In [2]:
toposcale_path = '/Users/jacobqs/Documents/MST Himalaya/TopoScale/TopoSCALE_BG.nc'

In [3]:
cell_data_path = '/Users/jacobqs/Documents/MST Himalaya/WFDE5/Cell_Data/cell_data.nc'

In [4]:
ds = xr.open_mfdataset(toposcale_path, engine='netcdf4')

In [5]:
ds

<xarray.Dataset>
Dimensions:  (t: 149016, clusters: 200, pixels: 571612, latitude: 1560,
              longitude: 1380)
Dimensions without coordinates: t, clusters, pixels, latitude, longitude
Data variables: (12/15)
    T        (t, clusters) int16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    U        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    q        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    LW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    SW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    P        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    ...       ...
    latp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lonp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    cn       (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    latg     (latitude) float64 dask.array<chunksize=(1560,), meta=np.ndarray>
    long     (longitude) float64 dask.array<chunksize=(1380,), meta=np.ndarray>
    mask     (longitude, latitude) uint8 dask.array<chunksize=(1380, 1560), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [6]:
cn = ds.cn.values.astype('int') -1 

In [7]:
ds_cn = ds.sel(clusters = cn)

/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [8]:
ds_cn = ds_cn.drop_dims(['longitude', 'latitude'])

In [9]:
ds_cn

<xarray.Dataset>
Dimensions:  (t: 149016, clusters: 571612, pixels: 571612)
Dimensions without coordinates: t, clusters, pixels
Data variables:
    T        (t, clusters) int16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    U        (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    q        (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    LW       (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    SW       (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    P        (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    ps       (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    latc     (clusters) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lonc     (clusters) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    latp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lonp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    cn       (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [10]:
# Create new variables using assign()
ds_cn = ds_cn.assign(cn_new=ds['cn'], lon=ds['lonp'], lat=ds['latp'])

# Rename the dimensions of the new variables
ds_cn = ds_cn.rename({'pixels': 'clusters'})

In [11]:
ds_cn = ds_cn.drop_vars(['cn', 'lonp', 'latp', 'latc', 'lonc'])

In [12]:
ds_cn = ds_cn.rename({'cn_new' : 'cn', 't' : 'time'})

In [13]:
ds_cn

<xarray.Dataset>
Dimensions:  (time: 149016, clusters: 571612)
Dimensions without coordinates: time, clusters
Data variables:
    T        (time, clusters) int16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    U        (time, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    q        (time, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    LW       (time, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    SW       (time, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    P        (time, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    ps       (time, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    cn       (clusters) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lon      (clusters) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lat      (clusters) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [14]:
# reshape data to have dimensions (t, latp, lonp)
ds_cn = ds_cn.set_index(clusters = ['lon',"lat"])
ds_cn = ds_cn.unstack("clusters")

/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return 

In [15]:
ds_cn

<xarray.Dataset>
Dimensions:  (lon: 912, lat: 1111, time: 149016)
Coordinates:
  * lon      (lon) float64 84.45 84.45 84.45 84.45 ... 85.2 85.2 85.2 85.21
  * lat      (lat) float64 28.01 28.01 28.01 28.01 ... 28.93 28.94 28.94 28.94
Dimensions without coordinates: time
Data variables:
    T        (time, lon, lat) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    U        (time, lon, lat) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    q        (time, lon, lat) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    LW       (time, lon, lat) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    SW       (time, lon, lat) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    P        (time, lon, lat) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    ps       (time, lon, lat) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    cn       (lon, lat) float64 dask.array<chunksize=(912, 1111), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [ ]:
ds_cell_data = xr.open_mfdataset(cell_data_path, engine='netcdf4')

In [ ]:
budhi_gandaki_ds = ds_cell_data.where(ds_cell_data.catchment_id == 10, drop = True).copy()

In [ ]:
x = budhi_gandaki_ds.x.values
y = budhi_gandaki_ds.y.values

In [ ]:
## Making a pyproj object for UTM Zone 45N

crs_4326 = CRS.from_epsg(4326) # Lat-lon
crs_32645 = CRS.from_epsg(32645) # CRS 

## Create transformer to convert from CRS to CRS

transformer = Transformer.from_crs(crs_32645, crs_4326, always_xy = True)

lon_bg, lat_bg = transformer.transform(x, y)

In [ ]:
ds_cn.isel(lat=0, time = 0).T.values

: 

: 

In [ ]:
lon_bg.sort()
lat_bg.sort()

In [ ]:
ds_out = xr.Dataset(
    {
        "lat": (["lon"], np.arange(np.min(lon_bg), np.max(lon_bg), 0.01), {"units": "degrees_north"}),
        "lon": (["lat"], np.arange(np.min(lat_bg), np.max(lat_bg), 0.01), {"units": "degrees_east"}),
    }
)
ds_out

<xarray.Dataset>
Dimensions:  (lon: 73, lat: 90)
Coordinates:
    lat      (lon) float64 84.46 84.47 84.48 84.49 ... 85.15 85.16 85.17 85.18
    lon      (lat) float64 28.03 28.04 28.05 28.06 ... 28.89 28.9 28.91 28.92
Data variables:
    *empty*

In [22]:
regridder = xe.Regridder(ds_cn, ds_out, 'conservative')

In [23]:
dr_out = regridder(ds_cn, keep_attrs=True)

In [25]:
dr_out.T

<xarray.DataArray 'T' (time: 149016, lon: 73, lat: 90)>
dask.array<_regrid, shape=(149016, 73, 90), dtype=float64, chunksize=(149016, 73, 90), chunktype=numpy.ndarray>
Coordinates:
    lat      (lon) float64 84.46 84.47 84.48 84.49 ... 85.15 85.16 85.17 85.18
    lon      (lat) float64 28.03 28.04 28.05 28.06 ... 28.89 28.9 28.91 28.92
Dimensions without coordinates: time
Attributes:
    long_name:  2 m air temperature [deg C]. Scale factor = 0.01

In [14]:
# Select only the `cn` variable
ds_cn_new = ds['cn']

In [15]:
# Merge the two datasets along the `pixels` dimension
ds_merged = xr.merge([ds_cn, ds_cn_new], join='inner')

In [17]:
# Transpose the dimensions to have `lonp` and `latp` as the first two dimensions
ds_transposed = ds_merged.transpose('t', 'lonp', 'latp', ...)

In [19]:
# Interpolate `cn` onto the same grid as `T`
ds_cn_interp = ds_transposed.interp(lonp=ds_transposed['lonp'], latp=ds_transposed['latp'])

In [22]:
# Merge the two datasets along the `pixels` dimension
ds_merged = xr.merge([ds_transposed, ds_cn_interp.rename({'pixels':'cn'})])

: 

: 

In [11]:
ds_merged

<xarray.Dataset>
Dimensions:  (lonp: 912, latp: 1111, t: 149016)
Coordinates:
  * lonp     (lonp) float64 84.45 84.45 84.45 84.45 ... 85.2 85.2 85.2 85.21
  * latp     (latp) float64 28.01 28.01 28.01 28.01 ... 28.93 28.94 28.94 28.94
Dimensions without coordinates: t
Data variables:
    T        (t, lonp, latp) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    U        (t, lonp, latp) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    q        (t, lonp, latp) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    LW       (t, lonp, latp) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    SW       (t, lonp, latp) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    P        (t, lonp, latp) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    ps       (t, lonp, latp) float64 dask.array<chunksize=(149016, 912, 1111), meta=np.ndarray>
    latc     (lonp, latp) float64 dask.array<chunksize=(912, 1111), meta=np.ndarray>
    lonc     (lonp, latp) float64 dask.array<chunksize=(912, 1111), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [12]:
ds.cn

<xarray.DataArray 'cn' (pixels: 571612)>
dask.array<open_dataset-df6cf0bb92ce90a3821ddacdf08e1fe3cn, shape=(571612,), dtype=float64, chunksize=(571612,), chunktype=numpy.ndarray>
Dimensions without coordinates: pixels
Attributes:
    long_name:  Cluster number of each pixel

In [16]:
# Select the relevant variables
ds_new = ds_cn[['T', 'U', 'SW', 'q', 'P', 'ps', 'latp', 'lonp', 't', 'clusters']]

In [17]:
# Set the index to 't' and 'clusters'
ds_new = ds_new.set_index({'t': 't', 'clusters': 'clusters'})


In [20]:
ds_new

<xarray.Dataset>
Dimensions:   (t: 149016, clusters: 571612, pixels: 571612)
Coordinates:
  * t         (t) int64 0 1 2 3 4 5 ... 149011 149012 149013 149014 149015
  * clusters  (clusters) int64 0 1 2 3 4 ... 571607 571608 571609 571610 571611
Dimensions without coordinates: pixels
Data variables:
    T         (t, clusters) int16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    U         (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    SW        (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    q         (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    P         (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    ps        (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    latp      (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lonp      (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [19]:
# Unstack 'latp' and 'lonp' to create new dimensions
ds_new = ds_new.unstack('latp')
ds_new = ds_new.unstack('lonp')


ValueError: Dataset does not contain the dimensions: ['latp']

In [16]:
new_dataset = ds_cn[['T', 'U', 'SW', 'q', 'P', 'ps']].sel(lonp=slice(None), latp=slice(None)).transpose('t', 'lonp', 'latp')


KeyError: "'lonp' is not a valid dimension or coordinate"

In [10]:
# create new dataset with x and y coordinates
new_dataset = xr.Dataset(coords={'lon': ds_cn.lonp, 'lat': ds_cn.latp})

In [11]:
ds_cn

<xarray.Dataset>
Dimensions:  (t: 149016, clusters: 571612, pixels: 571612, latitude: 1560,
              longitude: 1380)
Dimensions without coordinates: t, clusters, pixels, latitude, longitude
Data variables: (12/15)
    T        (t, clusters) int16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    U        (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    q        (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    LW       (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    SW       (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    P        (t, clusters) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    ...       ...
    latp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lonp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    cn       (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    latg     (latitude) float64 dask.array<chunksize=(1560,), meta=np.ndarray>
    long     (longitude) float64 dask.array<chunksize=(1380,), meta=np.ndarray>
    mask     (longitude, latitude) uint8 dask.array<chunksize=(1380, 1560), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [12]:
# define function to map data onto new dataset
def map_data(data, x, y, cn):
    # perform mapping here
    return mapped_data

In [13]:
new_dataset

<xarray.Dataset>
Dimensions:  (pixels: 571612)
Coordinates:
    lon      (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lat      (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
Dimensions without coordinates: pixels
Data variables:
    *empty*

In [25]:
# apply function to Xarray dataset
mapped_data = xr.apply_ufunc(
    map_data, 
    ds_cn['T'], 
    new_dataset['lon'], 
    new_dataset['lat'], 
    ds_cn['clusters'], 
    input_core_dims=[['t', 'clusters'], ['pixels'], ['pixels'], ['clusters']], 
    output_core_dims=[['t', 'lon', 'lat']], 
    dask='parallelized',
    output_dtypes=[ds_cn['T'].dtype],
    output_sizes={'lon': len(ds.lonp), 'lat': len(ds.latp)}
)

/var/folders/07/3pj74xcs7mqdgyzchd88y9840000gn/T/ipykernel_74558/2555894829.py:2: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  mapped_data = xr.apply_ufunc(


In [26]:
mapped_data_ds = mapped_data.to_dataset(name='T')

In [36]:
mapped_data_ds = mapped_data_ds.update(dict(lon = ds.lonp, lat = ds.latp))

In [37]:
mapped_data_ds = mapped_data_ds.assign(cn = ds_cn.cn)

In [38]:
mapped_data_ds

<xarray.Dataset>
Dimensions:  (t: 149016, lon: 571612, lat: 571612, pixels: 571612)
Coordinates:
    lon      (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lat      (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
Dimensions without coordinates: t, pixels
Data variables:
    T        (t, lon, lat) int16 dask.array<chunksize=(149016, 571612, 571612), meta=np.ndarray>
    cn       (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>

In [10]:
# Select desired variables and dimensions
new_ds = ds[['T', 'U', 'ps', 'P', 'SW', 'q']].sel(lonp=ds.lonp, latp=ds.latp).drop_dims(['clusters', 'pixels'])

KeyError: "'lonp' is not a valid dimension or coordinate"

In [ ]:
ds_copy = ds.copy()

## Making a pyproj object for UTM Zone 45N

crs_4326 = CRS.from_epsg(4326) # Lat-lon
crs_32645 = CRS.from_epsg(32645) # CRS 

## Create transformer to convert from CRS to CRS

transformer = Transformer.from_crs(crs_4326, crs_32645, always_xy = True)

x, y = transformer.transform(lonp, latp)

x = xr.DataArray(x, dims='pixels')
y = xr.DataArray(y, dims='pixels')

# Assign the new coordinate variables to the dataset
ds_copy = ds_copy.assign_coords(x=x, y=y)


In [ ]:
ds_copy

<xarray.Dataset>
Dimensions:  (t: 149016, clusters: 200, pixels: 571612, latitude: 1560,
              longitude: 1380)
Coordinates:
    x        (pixels) float64 2.507e+05 2.507e+05 ... 3.245e+05 3.245e+05
    y        (pixels) float64 3.185e+06 3.185e+06 ... 3.167e+06 3.167e+06
Dimensions without coordinates: t, clusters, pixels, latitude, longitude
Data variables: (12/15)
    T        (t, clusters) int16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    U        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    q        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    LW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    SW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    P        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    ...       ...
    latp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lonp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    cn       (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    latg     (latitude) float64 dask.array<chunksize=(1560,), meta=np.ndarray>
    long     (longitude) float64 dask.array<chunksize=(1380,), meta=np.ndarray>
    mask     (longitude, latitude) uint8 dask.array<chunksize=(1380, 1560), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [11]:
cn = ds.cn.values.astype('int') -1 
T_cn = ds.T.sel(clusters = cn)
U_cn = ds.U.sel(clusters = cn)
q_cn = ds.q.sel(clusters = cn)
SW_cn = ds.SW.sel(clusters = cn)
P_cn = ds.P.sel(clusters = cn)
ps_cn = ds.ps.sel(clusters = cn)
lonp = ds.lonp.values
latp = ds.latp.values
t = ds.t.values
clusters = T_cn.clusters.values


/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [12]:
df = pd.DataFrame()
df['lonp'] = lonp
df['latp'] = latp
df['cn'] = cn

## Making a pyproj object for UTM Zone 45N

crs_4326 = CRS.from_epsg(4326) # Lat-lon
crs_32645 = CRS.from_epsg(32645) # CRS 

## Create transformer to convert from CRS to CRS

transformer = Transformer.from_crs(crs_4326, crs_32645, always_xy = True)

x, y = transformer.transform(lonp, latp)

df['x'] = x
df['y'] = y

# # Extracting only data within the area of interest
# x_min = np.min(x)
# x_max = np.max(x)
# y_min = np.min(y)
# y_max = np.max(y)

# x_min_grid = x_min - 500
# x_max_grid = x_max + 500
# y_min_grid = y_min - 500
# y_max_grid = y_max + 500

# x_grid = np.arange(x_min_grid, x_max_grid, 1000)
# y_grid = np.arange(y_min_grid, y_max_grid, 1000)

# df['x_grid'] = np.nan
# df['y_grid'] = np.nan

# # for i, val in enumerate(df['x']):
# #     x_dist = np.abs(df['x'][i] - x_grid)
# #     x_min_dist_inx = np.where(x_dist == np.min(x_dist))[0]
# #     df.loc[i, 'x_grid'] = x_grid[x_min_dist_inx]
# #     y_dist = np.abs(df['y'][i] - y_grid)
# #     y_min_dist_inx = np.where(y_dist == np.min(y_dist))[0]
# #     df.loc[i, 'y_grid'] = y_grid[y_min_dist_inx]

In [91]:
ds_new = ds.assign_coords(dict(x = (['pixels'], x), y = (['pixels'], y), cn = (['pixels'], cn)))

In [92]:
ds_new

<xarray.Dataset>
Dimensions:  (t: 149016, clusters: 200, pixels: 571612, latitude: 1560,
              longitude: 1380)
Coordinates:
    cn       (pixels) int64 103 103 103 103 43 58 58 ... 88 11 88 59 42 42 132
    x        (pixels) float64 2.507e+05 2.507e+05 ... 3.245e+05 3.245e+05
    y        (pixels) float64 3.185e+06 3.185e+06 ... 3.167e+06 3.167e+06
Dimensions without coordinates: t, clusters, pixels, latitude, longitude
Data variables: (12/14)
    T        (t, clusters) int16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    U        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    q        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    LW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    SW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    P        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    ...       ...
    lonc     (clusters) float64 dask.array<chunksize=(200,), meta=np.ndarray>
    latp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lonp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    latg     (latitude) float64 dask.array<chunksize=(1560,), meta=np.ndarray>
    long     (longitude) float64 dask.array<chunksize=(1380,), meta=np.ndarray>
    mask     (longitude, latitude) uint8 dask.array<chunksize=(1380, 1560), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [10]:
ds_new = ds_new.assign(dict(T_cn = (['t', 'cn'], T_cn.data), U_cn = (['t', 'cn'], U_cn.data),
                            q_cn = (['t', 'cn'], q_cn.data), SW_cn = (['t', 'cn'], SW_cn.data),
                            P_cn = (['t', 'cn'], P_cn.data), ps_cn = (['t', 'cn'], ps_cn.data)))

In [11]:
ds_new

<xarray.Dataset>
Dimensions:  (t: 149016, clusters: 200, pixels: 571612, latitude: 1560,
              longitude: 1380, cn: 571612)
Coordinates:
    cn       (pixels) int64 103 103 103 103 43 58 58 ... 88 11 88 59 42 42 132
    x        (pixels) float64 2.507e+05 2.507e+05 ... 3.245e+05 3.245e+05
    y        (pixels) float64 3.185e+06 3.185e+06 ... 3.167e+06 3.167e+06
Dimensions without coordinates: t, clusters, pixels, latitude, longitude
Data variables: (12/20)
    T        (t, clusters) int16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    U        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    q        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    LW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    SW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    P        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    ...       ...
    T_cn     (t, cn) int16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    U_cn     (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    q_cn     (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    SW_cn    (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    P_cn     (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    ps_cn    (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [12]:
ds_new = ds_new.drop_dims(['clusters', 'pixels', 'latitude', 'longitude'])

In [13]:
ds_new = ds_new.assign_coords(dict(x = (['cn'], x), y = (['cn'], y), cn = (['cn'], cn)))

In [14]:
# create new dataset with x and y coordinates
new_dataset = xr.Dataset(coords={'x': x, 'y': y})

In [15]:
# define function to map data onto new dataset
def map_data(data, x, y, cn):
    # perform mapping here
    return mapped_data

In [16]:
# apply function to Xarray dataset
mapped_data = xr.apply_ufunc(
    map_data, 
    ds_new['T_cn'], 
    new_dataset['x'], 
    new_dataset['y'], 
    ds_new['cn'], 
    input_core_dims=[['t', 'cn'], ['x'], ['y'], ['cn']], 
    output_core_dims=[['t', 'x', 'y']], 
    dask='parallelized',
    output_dtypes=[ds_new['T_cn'].dtype],
    output_sizes={'x': len(x), 'y': len(y)}
)

/var/folders/07/3pj74xcs7mqdgyzchd88y9840000gn/T/ipykernel_60680/440167258.py:2: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  mapped_data = xr.apply_ufunc(


In [17]:
mapped_data

<xarray.DataArray (t: 149016, x: 571612, y: 571612)>
dask.array<transpose, shape=(149016, 571612, 571612), dtype=int16, chunksize=(149016, 571612, 571612), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 2.507e+05 2.507e+05 2.507e+05 ... 3.245e+05 3.245e+05
  * y        (y) float64 3.185e+06 3.185e+06 3.185e+06 ... 3.167e+06 3.167e+06
Dimensions without coordinates: t

In [18]:
ds = xr.Dataset(
    {
        'mapped_data': mapped_data,
        'T_cn' : ds_new['T_cn'],
    },
    coords={
        't': ds_new['t'],
        'x': new_dataset['x'],
        'y': new_dataset['y'],
        'cn': ds_new['cn'],
    },
)


In [19]:
ds

<xarray.Dataset>
Dimensions:      (x: 571612, y: 571612, t: 149016, cn: 571612)
Coordinates:
  * x            (x) float64 2.507e+05 2.507e+05 ... 3.245e+05 3.245e+05
  * y            (y) float64 3.185e+06 3.185e+06 ... 3.167e+06 3.167e+06
  * t            (t) int64 0 1 2 3 4 5 ... 149011 149012 149013 149014 149015
  * cn           (cn) int64 103 103 103 103 43 58 58 ... 88 11 88 59 42 42 132
Data variables:
    mapped_data  (t, x, y) int16 dask.array<chunksize=(149016, 571612, 571612), meta=np.ndarray>
    T_cn         (t, cn) int16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>

In [44]:
ds

<xarray.Dataset>
Dimensions:  (t: 149016, clusters: 200, pixels: 571612, latitude: 1560,
              longitude: 1380)
Dimensions without coordinates: t, clusters, pixels, latitude, longitude
Data variables: (12/15)
    T        (t, clusters) int16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    U        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    q        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    LW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    SW       (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    P        (t, clusters) uint16 dask.array<chunksize=(149016, 200), meta=np.ndarray>
    ...       ...
    latp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    lonp     (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    cn       (pixels) float64 dask.array<chunksize=(571612,), meta=np.ndarray>
    latg     (latitude) float64 dask.array<chunksize=(1560,), meta=np.ndarray>
    long     (longitude) float64 dask.array<chunksize=(1380,), meta=np.ndarray>
    mask     (longitude, latitude) uint8 dask.array<chunksize=(1380, 1560), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [41]:
# compute number of grid cells in each dimension
num_x_cells = round((ds['x'].max().item() - ds['x'].min().item()) / 1000) + 1
num_y_cells = round((ds['y'].max().item() - ds['y'].min().item()) / 1000) + 1


# create new x and y coordinates with spacing of 1000
new_x = np.arange(ds['x'].min(), ds['x'].max()+1000, 1000)
new_y = np.arange(ds['y'].min(), ds['y'].max()+1000, 1000)

# regrid dataset with new coordinates
new_ds = ds.reindex(x=new_x, y=new_y)

# compute cell area
cell_area = (new_x[1] - new_x[0]) * (new_y[1] - new_y[0])

# reshape x, y, and cn arrays
x = new_ds['x'].values.reshape(-1)
y = new_ds['y'].values.reshape(-1)
cn = new_ds['cn'].values.reshape(-1)

# group cn variable into cells based on new coordinates and count values in each cell
cn_counts, _, _ = np.histogram2d(x, y, bins=[new_x, new_y], weights=cn)

# convert to xarray DataArray and drop last bin of new_x and new_y
cn_counts = xr.DataArray(cn_counts, coords={'x': new_x[:-1], 'y': new_y[:-1]}, dims=['x', 'y'])

# compute proportion of cn values within each cell
prop_cn = cn_counts / (cell_area * len(ds['t']))


# compute total count of cn values
total_count = cn_counts.sum(dim=['x', 'y'])

# compute proportion of cn values in each cell
cn_prop = cn_counts / total_count

# compute weighted average of mapped_data variable
mapped_data_weighted_avg = (new_ds['mapped_data'] * new_ds['T_cn'] * cell_area * cn_prop).sum(dim=['x', 'y', 'cn']) / (new_ds['T_cn'] * cell_area * cn_prop).sum(dim=['x', 'y', 'cn'])


/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1380: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


ValueError: x and y must have the same length.

In [32]:
new_ds

<xarray.Dataset>
Dimensions:      (t: 149016, x: 75, y: 105, cn: 571612)
Coordinates:
  * t            (t) int64 0 1 2 3 4 5 ... 149011 149012 149013 149014 149015
  * cn           (cn) int64 103 103 103 103 43 58 58 ... 88 11 88 59 42 42 132
  * x            (x) float64 2.506e+05 2.516e+05 ... 3.236e+05 3.246e+05
  * y            (y) float64 3.1e+06 3.101e+06 3.102e+06 ... 3.203e+06 3.204e+06
Data variables:
    mapped_data  (t, x, y) float64 dask.array<chunksize=(149016, 75, 105), meta=np.ndarray>
    T_cn         (t, cn) int16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>

In [25]:
# Compute the weights as the proportion of 'cn' values in each cell in the coarser grid
coarse_ds = new_dataset.coarsen(x=1000, y=1000, boundary='trim').mean()
weights = coarse_ds.cn.count() / new_dataset.cn.count()
weights = np.expand_dims(weights, axis=-1) # add 'cn' dimension to weights

# Apply the weighted mean function to each cell in the coarser grid
regridded_ds = xr.apply_ufunc(
    weighted_mean,
    new_dataset.mapped_data,
    weights,
    input_core_dims=[['cn'], ['cn']],
    output_core_dims=[['t', 'x', 'y']],
    dask='parallelized',
    output_dtypes=[new_dataset.mapped_data.dtype]
)

# Assign the new variable to the dataset
ds['regridded_data'] = regridded_ds

ValueError: operand to apply_ufunc has required core dimensions ['cn'], but some of these dimensions are absent on an input variable: ['cn']

In [23]:
# Example dataset with coordinates x, y, and t
ds = xr.Dataset({
    'data': (['t', 'x', 'y'], new_dataset.mapped_data.data),
    'T_cn': (['t', 'cn'], ds_new.T_cn.data),
})

In [26]:
# Add x and y coordinates to the dataset
ds.coords['x'] = x
ds.coords['y'] = y

# Define the desired spacing for the regridding
dx = 1000
dy = 1000

# Create a MultiIndex with the original coordinate values
mi = ds.set_index(cn=['x', 'y']).coords['cn'].to_index()

ValueError: unmatched dimensions for multi-index variables 'x' ('x',), 'y' ('y',)

In [41]:
new_x = np.arange(new_dataset['x'].min(), new_dataset['x'].max(), 1000)
new_y = np.arange(new_dataset['y'].min(), new_dataset['y'].max(), 1000)

In [48]:
unique_vals_x, unique_idx_x = np.unique(new_x, return_index=True)
unique_vals_y, unique_idx_y = np.unique(new_y, return_index=True)

In [51]:
unique_arr_x = new_x[unique_idx_x]
unique_arr_y = new_y[unique_idx_y]

In [52]:
coarse_dataset = new_dataset.interp(x=new_x, y=new_y, method='nearest')


/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1380: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [15]:
cell_data_ds = xr.open_dataset(cell_data_path)
cell_data_ds = cell_data_ds.where(cell_data_ds.catchment_id == 10, drop = True).copy()
x = cell_data_ds.x.values
y = cell_data_ds.y.values

In [16]:
crs_4326 = CRS.from_epsg(4326) # Lat-lon
crs_32645 = CRS.from_epsg(32645) # CRS 

## Create transformer to convert from CRS to CRS

transformer = Transformer.from_crs( crs_32645, crs_4326, always_xy = True)

lon, lat = transformer.transform(x, y)


In [17]:
xmin = np.min(x)
xmax = np.max(x)
ymin = np.min(y)
ymax = np.max(y)

In [18]:
crs = '+proj=utm +zone=45 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.x, df.y),
                       crs = crs)
xmin, ymin, xmax, ymax = gdf.total_bounds

In [19]:
cell_data_ds = xr.open_dataset(cell_data_path)
cell_data_ds = cell_data_ds.where(cell_data_ds.catchment_id == 10, drop = True).copy()

In [20]:
xmin = cell_data_ds.x.min().values
xmax = cell_data_ds.x.max().values
ymin = cell_data_ds.y.min().values
ymax = cell_data_ds.y.max().values

In [21]:
# create the cells in a loop
cell_size = 5000
grid_cells = []
xs = []
ys = []
for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
    for y0 in np.arange(ymin, ymax+cell_size, cell_size):
        # bounds
        x1 = x0-cell_size
        y1 = y0+cell_size
        grid_cells.append(shapely.geometry.box(x0, y0, x1, y1))
        xs.append((x1-x0)/2 + x0)
        ys.append((y1-y0)/2 + y0)
cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
                                 crs=crs)

In [22]:
cell

,geometry
0,"POLYGON ((246933.926 3102291.499, 246933.926 3..."
1,"POLYGON ((246933.926 3107291.499, 246933.926 3..."
2,"POLYGON ((246933.926 3112291.499, 246933.926 3..."
3,"POLYGON ((246933.926 3117291.499, 246933.926 3..."
4,"POLYGON ((246933.926 3122291.499, 246933.926 3..."
...,...
331,"POLYGON ((321933.926 3182291.499, 321933.926 3..."
332,"POLYGON ((321933.926 3187291.499, 321933.926 3..."
333,"POLYGON ((321933.926 3192291.499, 321933.926 3..."
334,"POLYGON ((321933.926 3197291.499, 321933.926 3..."


In [23]:
# ax = gdf.plot(markersize=.1, figsize=(12, 8), column='cn', cmap='jet')
# plt.autoscale(False)
# cell.plot(ax=ax, facecolor="none", edgecolor='grey')
# ax.axis("off")

In [24]:
merged = gpd.sjoin(gdf, cell, how='left', op='within')

/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [25]:
merged_copy = merged.copy()

In [26]:
# Group merged_copy by the index_right column and count the number of cn values in each group
grouped_df = merged_copy.groupby(['index_right', 'cn']).count()

In [27]:
# add a column to the grouped_df with the sum of the counts for each group
grouped_df['total'] = grouped_df.groupby(level=0)['geometry'].transform('sum')

In [28]:
grouped_df['proportion'] = grouped_df['geometry'] / grouped_df['total']

In [29]:
# Make a list with the first elements in the list of tuples in grouped_df.index
index_right = [i[0] for i in grouped_df.index]
grouped_cn = [i[1] for i in grouped_df.index]

In [30]:
proportions = grouped_df['proportion'].values

In [31]:
grouped_df['n'] = np.arange(len(grouped_df))

In [32]:
grouped_df_copy = grouped_df.copy()

In [33]:
grouped_cn

[33,
 103,
 4,
 9,
 14,
 28,
 33,
 34,
 43,
 52,
 53,
 54,
 58,
 64,
 67,
 74,
 91,
 103,
 109,
 141,
 144,
 157,
 158,
 165,
 166,
 191,
 193,
 196,
 53,
 54,
 72,
 103,
 138,
 158,
 193,
 4,
 9,
 14,
 17,
 33,
 34,
 35,
 36,
 43,
 53,
 58,
 64,
 67,
 70,
 71,
 72,
 74,
 84,
 91,
 106,
 109,
 112,
 134,
 138,
 140,
 141,
 144,
 158,
 160,
 163,
 164,
 165,
 168,
 169,
 176,
 181,
 191,
 193,
 196,
 4,
 9,
 14,
 24,
 28,
 33,
 34,
 36,
 43,
 52,
 53,
 54,
 58,
 64,
 67,
 74,
 91,
 103,
 107,
 109,
 112,
 134,
 138,
 141,
 144,
 157,
 158,
 160,
 163,
 164,
 165,
 166,
 191,
 193,
 196,
 4,
 9,
 17,
 28,
 33,
 34,
 35,
 36,
 43,
 52,
 53,
 54,
 61,
 64,
 71,
 72,
 74,
 84,
 91,
 97,
 103,
 106,
 107,
 109,
 112,
 129,
 134,
 138,
 140,
 141,
 144,
 157,
 158,
 160,
 163,
 164,
 165,
 168,
 176,
 181,
 184,
 187,
 191,
 193,
 196,
 4,
 9,
 28,
 33,
 34,
 35,
 36,
 43,
 53,
 54,
 58,
 61,
 64,
 67,
 74,
 84,
 91,
 103,
 107,
 109,
 129,
 134,
 138,
 141,
 144,
 157,
 158,
 164,
 165,
 184

In [34]:
grouped_df_copy

lonp  latp   x   y  geometry  total  proportion      n
index_right cn                                                         
15.0        33     38    38  38  38        38     45    0.844444      0
            103     7     7   7   7         7     45    0.155556      1
16.0        4       9     9   9   9         9    593    0.015177      2
            9       2     2   2   2         2    593    0.003373      3
            14     70    70  70  70        70    593    0.118044      4
...               ...   ...  ..  ..       ...    ...         ...    ...
328.0       170     1     1   1   1         1   1103    0.000907  11620
            171    16    16  16  16        16   1103    0.014506  11621
            179     3     3   3   3         3   1103    0.002720  11622
            195     4     4   4   4         4   1103    0.003626  11623
            199    11    11  11  11        11   1103    0.009973  11624

[11625 rows x 8 columns]

In [35]:
T_merged = ds.T.sel(clusters = grouped_cn)

/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [36]:
# Reshape T_merged into the dimensions t, index_right, group_cn
T_merged.clusters

<xarray.DataArray 'clusters' (clusters: 11625)>
array([    0,     1,     2, ..., 11622, 11623, 11624])
Dimensions without coordinates: clusters

In [37]:
grouped_df_copy = grouped_df.copy()

In [38]:
flattend_df = grouped_df_copy.groupby('index_right').count()

In [39]:
flattend_df

,lonp,latp,x,y,geometry,total,proportion,n
index_right,,,,,,,,
15.0,2,2,2,2,2,2,2,2
16.0,26,26,26,26,26,26,26,26
34.0,7,7,7,7,7,7,7,7
35.0,39,39,39,39,39,39,39,39
36.0,35,35,35,35,35,35,35,35
...,...,...,...,...,...,...,...,...
310.0,24,24,24,24,24,24,24,24
325.0,12,12,12,12,12,12,12,12
326.0,47,47,47,47,47,47,47,47


In [40]:
n_unique = np.arange(len(flattend_df.index))
n_unique

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [43]:
ds_new

<xarray.Dataset>
Dimensions:  (t: 149016, cn: 571612)
Coordinates:
    x        (cn) float64 2.507e+05 2.507e+05 2.507e+05 ... 3.245e+05 3.245e+05
    y        (cn) float64 3.185e+06 3.185e+06 3.185e+06 ... 3.167e+06 3.167e+06
  * cn       (cn) int64 103 103 103 103 43 58 58 165 ... 88 11 88 59 42 42 132
Dimensions without coordinates: t
Data variables:
    T_cn     (t, cn) int16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    U_cn     (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    q_cn     (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    SW_cn    (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    P_cn     (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
    ps_cn    (t, cn) uint16 dask.array<chunksize=(149016, 571612), meta=np.ndarray>
Attributes:
    name:              TopoSCALE_BG.nc
    contains:          Hourly "TopoSCALEd" ERA5 forcing at 90 m resolution fo...
    created by:        Kristoffer Aalstad
    created:           05-Jan-2023 21:35:51
    timestamps:        The dataare hourly starting on the 01.01.1999 00:00 UT...
    origin:            The data was TopoSCALed in Matlab using the TopoLAB to...
    acknowledgements:  Thanks to C3S & ECMWF for providing the ERA5 reanalysi...
    citation:          See Fiddes et al. (2022, https://doi.org/10.5194/gmd-1...
    contact:           kristoffer.aalstad@geo.uio.no for more information and...

In [45]:
# create new dataset with x and y coordinates
new_dataset = xr.Dataset(coords={'x': x, 'y': y})

In [46]:
# define function to map data onto new dataset
def map_data(data, x, y, cn):
    # perform mapping here
    return mapped_data

In [50]:
# apply function to Xarray dataset
mapped_data = xr.apply_ufunc(
    map_data, 
    ds_new['T_cn'], 
    new_dataset['x'], 
    new_dataset['y'], 
    ds_new['cn'], 
    input_core_dims=[['t', 'cn'], ['x'], ['y'], ['cn']], 
    output_core_dims=[['t', 'x', 'y']], 
    dask='parallelized',
    output_dtypes=[ds_new['T_cn'].dtype],
    output_sizes={'x': len(x), 'y': len(y)}
)

/var/folders/07/3pj74xcs7mqdgyzchd88y9840000gn/T/ipykernel_42054/440167258.py:2: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  mapped_data = xr.apply_ufunc(


In [52]:
mapped_data.isel(t = 0, x = 0, y = 0).values

KeyboardInterrupt: 

In [27]:
bins = flattend_df['n'].values

In [28]:
bin_list = [0]
count = 0
for i, value in enumerate(flattend_df['n']):
    if i == 0:
        bin_list.append(value)
        count = value
    else:
        count = count + value
        bin_list.append(count)
bin_list = np.array(bin_list)

In [33]:
T_merged_weighted = T_merged * proportions * 0.01

<xarray.DataArray 'clusters' (clusters: 11625)>
array([    0,     1,     2, ..., 11622, 11623, 11624])
Dimensions without coordinates: clusters

In [31]:
T_merged_weighted.sel(t=0).values

array([-1540.26666667,  -257.91111111,   -24.57166948, ...,
          -4.94741614,    -4.92112421,   -19.9655485 ])

In [31]:
T_binned = T_merged_weighted.groupby_bins('clusters', bins = bin_list).sum()

/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [32]:
T_binned

<xarray.DataArray 'T' (t: 149016, clusters_bins: 213)>
dask.array<transpose, shape=(149016, 213), dtype=float64, chunksize=(149016, 1), chunktype=numpy.ndarray>
Coordinates:
  * clusters_bins  (clusters_bins) object (0, 2] (2, 28] ... (11589, 11625]
Dimensions without coordinates: t

In [33]:
T_first = T_binned.isel(t=0, clusters_bins = 0)

In [34]:
T_first.values

KeyboardInterrupt: 

In [27]:
# Reshape T_merged_weighted such that clusters is replaced by groped_df.index 
T_merged_weighted.res

ValueError: Dimensions {'n', 'z'} do not exist. Expected one or more of ('t', 'clusters')

In [28]:
ds_merged = xr.Dataset({'T': (['t', 'indx'], T_merged_weighted.values)})

: 

: 

In [20]:
len(T_merged_weighted.sel(t=0))

65862

In [46]:
def get_regridded(merged, ds):
    regridded = dict()
    t = ds.t.values
    cn = ds.cn.values.astype('int') -1 
    for i in t:
        merged_copy = merged.copy()
        ds_slice = ds.sel(t = i)
        merged_copy['T'] = ds_slice.sel(clusters = cn).T * 0.01
        df_count = pd.DataFrame(merged_copy.groupby(['index_right', 'cn', 'T']).size()).rename(columns={0:'count'})
        df_count['proportion'] = df_count.groupby(level=0).transform(lambda x: x/x.sum())
        temps = []
        for j in range(len(df_count)):
            temp = df_count.index[j][2]
            temps.append(temp)
        df_count['temp'] = temps
        df_count['T_scaled'] = df_count['temp'] * df_count['proportion']
        temperature = df_count.groupby(level=0).sum('T_scaled')['T_scaled'].values
        regridded[i] = temperature
    return regridded

In [47]:
temp_regridded = get_regridded(merged, ds)

KeyboardInterrupt: 

In [50]:
T = ds.sel(clusters = cn).T * 0.01

/Users/jacobqs/opt/miniconda3/envs/geohyd/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [52]:
T

<xarray.DataArray 'T' (t: 149016, clusters: 571612)>
dask.array<mul, shape=(149016, 571612), dtype=float64, chunksize=(149016, 571612), chunktype=numpy.ndarray>
Dimensions without coordinates: t, clusters

In [53]:
iterables = [T.t.values, T.clusters.values]

In [56]:
pd.MultiIndex.from_product(iterables, names=["t", "clusters"])

: 

: 

In [51]:
df = pd.DataFrame()


<xarray.DataArray 'T' (t: 149016, clusters: 571612)>
dask.array<mul, shape=(149016, 571612), dtype=float64, chunksize=(149016, 571612), chunktype=numpy.ndarray>
Dimensions without coordinates: t, clusters

In [10]:
ds0 = ds.sel(t=0)

In [17]:
merged_copy['T'] = ds0.sel(clusters = cn).T * 0.01

In [18]:
merged_copy

,lonp,latp,cn,x,y,geometry,index_right,T
0,84.446250,28.772500,103,250677.618451,3.185457e+06,POINT (250677.618 3185456.567),190.0,-16.58
1,84.446250,28.771667,103,250675.635417,3.185364e+06,POINT (250675.635 3185364.194),189.0,-16.58
2,84.446250,28.770833,103,250673.652435,3.185272e+06,POINT (250673.652 3185271.821),189.0,-16.58
3,84.446250,28.770000,103,250671.669507,3.185179e+06,POINT (250671.670 3185179.449),189.0,-16.58
4,84.446250,28.769167,43,250669.686631,3.185087e+06,POINT (250669.687 3185087.076),189.0,-21.41
...,...,...,...,...,...,...,...,...
571607,85.205417,28.618333,88,324553.847873,3.167018e+06,POINT (324553.848 3167017.724),7836.0,-17.31
571608,85.205417,28.617500,59,324552.462054,3.166925e+06,POINT (324552.462 3166925.379),7836.0,-19.39
571609,85.205417,28.616667,42,324551.076273,3.166833e+06,POINT (324551.076 3166833.033),7836.0,-16.37
571610,85.205417,28.615833,42,324549.690529,3.166741e+06,POINT (324549.691 3166740.687),7836.0,-16.37


In [21]:
# Make dataframe that has the merged['index_right] and merged['cn'] as multindex, and the count of cn as the value
# Rename the column to 'count'  
df_count = pd.DataFrame(merged_copy.groupby(['index_right', 'cn', 'T']).size()).rename(columns={0:'count'})

In [22]:
df_count

count
index_right cn  T            
186.0       33  -18.24      3
            103 -16.58     10
187.0       14  -15.63      5
            33  -18.24     63
            34  -17.67      4
...                       ...
7839.0      110 -15.37      1
            133 -14.15      1
            135 -15.61     10
            152 -16.62      3
            171 -14.06      1

[65862 rows x 1 columns]

In [23]:
# Add a new column to df_count that is the percentage of the total count for each cluster
df_count['proportion'] = df_count.groupby(level=0).transform(lambda x: x/x.sum())

In [24]:
df_count

count  proportion
index_right cn  T                        
186.0       33  -18.24      3    0.230769
            103 -16.58     10    0.769231
187.0       14  -15.63      5    0.050505
            33  -18.24     63    0.636364
            34  -17.67      4    0.040404
...                       ...         ...
7839.0      110 -15.37      1    0.024390
            133 -14.15      1    0.024390
            135 -15.61     10    0.243902
            152 -16.62      3    0.073171
            171 -14.06      1    0.024390

[65862 rows x 2 columns]

In [31]:
temps = []
for i in range(len(df_count)):
    temp = df_count.index[i][2]
    temps.append(temp)


In [32]:
df_count['temp'] = temps

In [34]:
df_count

count  proportion   temp
index_right cn  T                               
186.0       33  -18.24      3    0.230769 -18.24
            103 -16.58     10    0.769231 -16.58
187.0       14  -15.63      5    0.050505 -15.63
            33  -18.24     63    0.636364 -18.24
            34  -17.67      4    0.040404 -17.67
...                       ...         ...    ...
7839.0      110 -15.37      1    0.024390 -15.37
            133 -14.15      1    0.024390 -14.15
            135 -15.61     10    0.243902 -15.61
            152 -16.62      3    0.073171 -16.62
            171 -14.06      1    0.024390 -14.06

[65862 rows x 3 columns]

In [35]:
df_count['T_scaled'] = df_count['temp'] * df_count['proportion']

In [40]:
temperature = df_count.groupby(level=0).sum('T_scaled')['T_scaled'].values